<div id="colab_button\">
    <h1>LaVague: Quick-tour guide</h1>
    <a target="_blank\" href="https://colab.research.google.com/github/lavague-ai/lavague/blob/main/docs/docs/get-started/quick-tour.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
    </div>

## Introduction

LaVague is an open-source framework allowing users to leverage AI to turn natural language instructions into executable code to automate UI actions, such as filling in a form, etc.

In this quick tour, we are going to show you how you can quickly get started with LaVague to:
- Launch an interactive Gradio where you can test running example actions on a webpage
- Generate the Python code needed to perform an action on the webpage of your choice based on natural language instructions 

> Pre-requisites: Note, if you are running the notebook locally, you will need Python (tested on python>=3.8) and pip installed.

## Installation

### Installing driver for Selenium

By default, LaVague generates code that leverages [Selenium](https://www.selenium.dev/) to perform user interface actions.

Selenium requires a driver to be installed to interface with the chosen browser (Chrome, Firefox, etc.)

We therefore first need to download the Chrome driver.

⚠️ For instructions on how to install a driver on a different OS, [see the Selenium documentation](https://selenium-python.readthedocs.io/installation.html#drivers)

> Note that while we use Selenium for this example. We hope to integrate different automation tools such as Playwright at a later date.

First, we need to make sure we have the following packages installed on our system.

> Note, if you are missing any of these packages, you will need to run `sudo apt update` first.

In [ ]:
# If you are missing any apt packages uncomment and run this command first:
# !sudo apt update

!sudo apt install -y ca-certificates fonts-liberation unzip \
libappindicator3-1 libasound2 libatk-bridge2.0-0 libatk1.0-0 libc6 \
libcairo2 libcups2 libdbus-1-3 libexpat1 libfontconfig1 libgbm1 \
libgcc1 libglib2.0-0 libgtk-3-0 libnspr4 libnss3 libpango-1.0-0 \
libpangocairo-1.0-0 libstdc++6 libx11-6 libx11-xcb1 libxcb1 \
libxcomposite1 libxcursor1 libxdamage1 libxext6 libxfixes3 libxi6 \
libxrandr2 libxrender1 libxss1 libxtst6 lsb-release wget xdg-utils

Now we can install the necessary Chrome webdriver files for Linux.

In [ ]:
!wget https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.94/linux64/chrome-linux64.zip
!wget https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.94/linux64/chromedriver-linux64.zip
!unzip chrome-linux64.zip
!unzip chromedriver-linux64.zip
!rm chrome-linux64.zip chromedriver-linux64.zip

Note that by default, LaVague assumes that the chrome-linux64 and chromedriver-linux64 folders will be available at root - we will therefore move them to the root directory

In [ ]:
!mv chrome-linux64/ ~/
!mv chromedriver-linux64/ ~/

### Installing LaVague

We now need to install LaVague, which contains the `ActionEngine` module dedicated to handling all the key AI operations and the `CommandCenter` module, which orchestrates the whole workflow, as well as the `LaVague CLI` tool.

> Note, here we will install LaVague from the latest GitHub commit on `main` to be sure we have the very latest files at this early fast-moving stage of the project. However, we do provide a PyPi package and will move to installing the package via pip once the project is more stable.

In [ ]:
!git clone https://github.com/lavague-ai/LaVague.git
!pip install -e LaVague

## Running LaVague with OpenAI's GPT3.5

### OpenAI API set up

Depending on the LLM we want to use, we need to set certain environment variables required for LaVague to work. 

⚠️ For this demo, we will use LaVague with GPT3.5 via the OpenAI API, so we will need to set our OpenAI key below for this example to work.

> If you don't have an OpenAI API key, you can see the OpenAI website for details [on how to create one](https://openai.com/product). Note, that this is a paid service - for a free alternative, see our [HuggingFace API integration](../integrations/hugging-face-api.ipynb)

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = # ADD YOUR OPENAI KEY HERE

### LaVague Launch

LaVague has two CLI commands `launch` and `build` - let's start by taking a look at the `launch` command which will launch an interactive Gradio where you can perform actions on your desired website.

The command we will use to do this is:
`lavague-launch --file_path hf.txt --config_path openai.py`

- We provide the `file_path` option with a text file containing the URL of the website we want to perform an action on and the natural language instructions for the action(s) we wish to perform.
-  We provide the `config_path` option with a Python file which configures LaVague for the specific LLM or set-up we wish to use.

We provide default configuration files for key integrations. 

Let's now download the default config files for OpenAI.

In [ ]:
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/examples/api/openai.py
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/tests/hf.txt

We can inspect the `openai.py` configuration file to see the default values we use by default here.

<img src="https://raw.githubusercontent.com/lavague-ai/LaVague/docs-add/docs/assets/openai_py.png" style="max-width: 50%;" />

We can now launch our interactive Gradio which will be created with three default instructions which can be executed on the HuggingFace website as defined in the `hf.txt` file.

In [ ]:
!lavague-launch --file_path hf.txt --config_path openai.py

You can now click on the public (if you are using Google Colab) or local URL to open the Gradio in your browser.

⚠️ Note, you will need to interact with the generated Gradio demo to perform the desired automated action. 

First, you should click in the URL textbox and press enter. Then, you should select your chosen default natural language instruction or write your own, and again click within the instruction textbox and press enter.

At this point, Selenium code in Python is generated by the LLM, which is then executed to perform the desired action on the website.

The action will then be visibly executed in the visual interface, but you can also check out the code LaVague generated and executed to perform this action on the right-hand side of the page.

### LaVague Build

We can alternatively use the `lavague-build` command to generate a Python script with the Selenium code needed to perform the desired action.

In [ ]:
!lavague-build --file_path hf.txt --config_path openai.py

This creates a script in your current directory named `hf_openai.py` - a combination of the two input file names separated by a `_` character.

This script contains the Python code using Selenium generated by the LLM to perform the desired action on the URL as specified in your `hf.txt` configuration file.

You can now inspect the code or execute it locally!

## Conclusions

That brings us to the end of this quick-tour. 

If you have any further questions, join us on the LaVague Discord [here](https://discord.com/invite/SDxn9KpqX9).